### In this notebook we guide you through an example how to get started with AESTETIK
1) Data loading and preprocessing
2) Morphology feature extraction
3) Training the model
4) Spot representations and clustering

In [ ]:
import os
os.chdir('../')

### Load required packages

In [ ]:
from torchvision.models import inception_v3, Inception_V3_Weights
from torchvision import transforms
import squidpy as sq
import scanpy as sc
import torch
import json

In [ ]:
from aestetik.utils.utils_morphology import extract_morphology_embeddings
from aestetik.utils.utils_transcriptomics import preprocess_adata
from aestetik.utils.utils_visualization import visualize
from aestetik import AESTETIK
AESTETIK.version()

In [ ]:
import logging
# Configure the logging module
logging.basicConfig(level=logging.INFO)  # Set the desired logging level
logging.getLogger("pyvips").setLevel(logging.CRITICAL)

### Load data and preprocess
We start by loading a sample of the `LIBD brain` dataset. This sample contains spatially resolved transcriptomics data and a corresponding high-resolution image.

In [ ]:
img_path = "test_data/151676.png"
adata_in = "test_data/151676.h5ad"
json_path = "test_data/151676.json"

In [ ]:
n_components = 15
spot_diameter_fullres = json.load(open(json_path))["spot_diameter_fullres"]
dot_size = json.load(open(json_path))["dot_size"]

print(f"spot_diameter_fullres: {spot_diameter_fullres}")
print(f"dot_size: {dot_size}")

In [ ]:
adata = sc.read_h5ad(adata_in)
#adata = adata[adata.obs.sample(100).index,:] # to speed up, we only select 100 spots.
adata = preprocess_adata(adata)

print(adata)

1) `x_array` and `y_array` contain the spatial coordinates of the spots on the array
2) `x_pixel` and `y_pixel`contain the spatial coordinates of the spots in the image
3) `ground_truth` - spot annotations

In [ ]:
adata.obs.head()

### Extract morphology features

In this example, we use Inception V3 to extract morphology spot features. For this, we use the last layer with dimension 2048. After that, we apply PCA to reduce the feature dimensions from 2048 to 15.

In [ ]:
weights = Inception_V3_Weights.DEFAULT
morphology_model = inception_v3(weights=weights)
morphology_model.fc = torch.nn.Identity()

morphology_model.eval()    
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
feature_dim = 2048

preprocess = transforms.Compose([
                transforms.ToTensor(),
                weights.transforms(antialias=True),
            ])

features_inception_v3 = extract_morphology_embeddings(img_path, 
                                             morphology_model,
                                             x_pixel=adata.obs.y_pixel, 
                                             y_pixel=adata.obs.x_pixel, 
                                             spot_diameter=spot_diameter_fullres,
                                             device=device,
                                             n_components=n_components,
                                             feature_dim=feature_dim,
                                             preprocess=preprocess,
                                             apply_pca=True)

We store the transcriptomics data in `X_pca_transcriptomics` and morphology data in `X_pca_morphology`.

In [ ]:
# we set the transcriptomics modality
adata.obsm["X_pca_transcriptomics"] = adata.obsm["X_pca"][:,0:n_components]

print(f"{adata.obsm['X_pca_transcriptomics'].shape[0]} spots x {adata.obsm['X_pca_transcriptomics'].shape[1]} transcriptomics PCA components")

In [ ]:
# we set the morphology modality
adata.obsm["X_pca_morphology"] = features_inception_v3[:,0:n_components]

print(f"{adata.obsm['X_pca_morphology'].shape[0]} spots x {adata.obsm['X_pca_morphology'].shape[1]} morphology PCA components")

### Model training

In our example, we set the morphology weight to 0 (no morphology contribution) and we refine the clusters in spatial space.

In [ ]:
parameters =    {'morphology_weight': 0,
                 'refine_cluster': True,
                 'window_size': 7
                }
parameters

In [ ]:
model = AESTETIK(nCluster=adata.obs.ground_truth.unique().size,
                 **parameters)

In [ ]:
model.fit(X=adata)

### Compute embeddings and clustering

In [ ]:
model.predict(X=adata, cluster=True)

In [ ]:
adata

Finally, spot representations can be found in `adata.obsm["AESTETIK"]` and clusters - in `adata.obs["AESTETIK_cluster"]`. We can visualize the clusters:

In [ ]:
sq.pl.spatial_scatter(adata, color=["ground_truth", 
                                    "X_pca_transcriptomics_cluster",
                                    "X_pca_morphology_cluster",
                                    "AESTETIK_cluster"], size=dot_size)

To analyze the model's performance during training, we can also plot the training loss: 

In [ ]:
visualize(model=model,
          plot_loss=True)